In [1]:
import matplotlib.pyplot as plt
import scipy.sparse
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.phrases import Phrases, Phraser
from nltk import bigrams
from sklearn.manifold import TSNE
import plotly.express as px
from gensim.models import Word2Vec

In [ ]:
! pip install  atlas
! pip install --upgrade scipy
! pip install gensim


In [3]:
df = pd.read_csv("threads_reviews.csv")
sentence = ""

for review in df["review_description"]:
    sentence += review

print(sentence)

Meh. Not the greatest experience on a Chromebook. Seems to be customized for phones only. Opens in a little screen that you can't expand or resize - for reasons that are a complete mystery to me. Judging from the fact that every other app I know of is resizeable, this seems like it was a conscious choice by the developers . Why you'd do something like this is beyond understanding and suggests a control freak approach. Not a great way to make a first impression.Pretty good for a first launch!! Its easy to use and self-explanatory and I'd say the algorithm is good as well. It has great *potential* but a few things need improvement: -the ability to use hashtags would make it easier to find topics and people you'd be interested in -an option for a following-only feed would be a major improvement (especially chronological order) -the ability to choose images from albums rather than recent photos -the ability to save photos to phoneFor a brand new app, it's very well optimized. However it's 

In [ ]:
x = sentence
with open('reviews.txt', 'w+') as fh:
    fh.write(x)

In [4]:
sent = [row.split() for row in df["review_description"]]
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [5]:
# Create bigram phrases
bigrams = Phrases(sent, min_count=30, progress_per=10000)
bigram_phraser = Phraser(bigrams)

In [6]:
sentences = list(bigram_phraser[sent])

In [7]:
# Tokenize the review_description column into sentences
sent = [row.split() for row in df["review_description"]]

# Create bigram phrases
bigrams = Phrases(sent, min_count=30, progress_per=10000)

# Convert the bigrams to a Phraser for better memory efficiency
bigram_phraser = Phraser(bigrams)

# Apply the bigram transformation to your sentences
sentences = list(bigram_phraser[sent])

# Train the Word2Vec model on the sentences (with bigram phrases)
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Get the word vectors (embeddings) for each word in the model's vocabulary
word_vectors = word2vec_model.wv

# Create a list of all unique words in the vocabulary
words = list(word_vectors.key_to_index.keys())

# Get the word embeddings for each word in the vocabulary
embeddings = np.array([word_vectors[word] for word in words])

# Reduce the word embeddings to 2D using t-SNE
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings)

# Create a DataFrame to store the word and its corresponding 2D embedding coordinates
df_tsne = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
df_tsne['word'] = words

# Create an interactive scatter plot using plotly
fig = px.scatter(df_tsne, x='x', y='y', text='word', hover_data=['word'], labels={'x': 't-SNE Dimension 1', 'y': 't-SNE Dimension 2'}, title='Word Embeddings Visualization')

# Add bigram phrases as annotations on the plot
for bigram in bigram_phraser.phrasegrams.keys():
    bigram_str = '_'.join(bigram)
    if bigram_str in words:
        x, y = df_tsne.loc[df_tsne['word'] == bigram_str, ['x', 'y']].values[0]
        fig.add_annotation(
            x=x,
            y=y,
            text=' '.join(bigram),
            showarrow=True,
            arrowhead=1,
            ax=0,
            ay=-40
        )

# Show the interactive plot
fig.show()